<a href="https://colab.research.google.com/github/Nov05/DS-Unit-2-Sprint-4-Project/blob/master/notebooks/02_plotly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# created by nov05 on 2019-08-01
# Nov05/DS-Unit-2-Sprint-4-Project/
# notebooks/

plotly example  https://colab.research.google.com/drive/1oijny-_J2K-7ma6jNq9jAp7C_tzNB27W  
https://medium.com/@plotlygraphs/plotly-py-4-0-is-here-offline-only-express-first-displayable-anywhere-fc444e5659ee  
https://community.plot.ly/t/introducing-plotly-py-4-0-0/25639  

https://plot.ly/python/figure-factory-subplots/

In [1]:
# !pip uninstall plotly
# !pip install plotly==3.6.1
!pip install plotly==4.0.0

     |████████████████████████████████| 6.8MB 2.8MB/s 
  Found existing installation: plotly 3.6.1
    Uninstalling plotly-3.6.1:
      Successfully uninstalled plotly-3.6.1


In [0]:
import plotly

# import plotly.plotly as py
## ImportError: 
## The plotly.plotly module is deprecated,
## please install the chart-studio package and use the
## chart_studio.plotly module instead. 

# import plotly.graph_objs as go # plotly 3.6.1
import plotly.graph_objects as go # plotly4.0.0
# plot = online mode, iplot = offline mode
from plotly.offline import init_notebook_mode, plot, iplot
from plotly.graph_objs import Histogram2dContour, Scatter
from plotly.graph_objs.histogram2dcontour import Contours
from plotly.graph_objs.scatter import Marker

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.figure_factory as ff

import plotly.tools as tls
from plotly.subplots import make_subplots

In [3]:
print("plotly", plotly.__version__)
# make sure it is version 4.0.0

plotly 4.0.0


In [0]:
# for plotly 4.0.0 you won't need this code block
# define this function for plotly and plotly express displaying
def configure_plotly_browser_state():
  import IPython
  display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
//// both would work, enjoy plotly and plotly express in colab
//               plotly: 'https://cdn.plot.ly/plotly-1.5.1.min.js?noext',
              plotly: 'https://cdn.plot.ly/plotly-latest.min.js?noext',
            },
          });
        </script>
        '''))
# init_notebook_mode(connected=True)

# then call the function configure_plotly_browser_state() 
# in EVERY code block where there is 
# a Plotly plot (each will be an iframe) 

# import and merge datasets

In [0]:
url_tasks = "https://raw.githubusercontent.com/Derek-Jones/SiP_dataset/master/Sip-task-info.csv"
url_dates = "https://raw.githubusercontent.com/Derek-Jones/SiP_dataset/master/est-act-dates.csv"
tasks  = pd.read_csv(url_tasks, encoding='iso-8859-1')
dates  = pd.read_csv(url_dates, encoding='iso-8859-1')

# merge two datasets
# there are duplicate TaskNumbers in both datasets
# there are duplicate rows in "dates" dataset
alldata = tasks.merge(right=dates[-dates.duplicated()], on='TaskNumber', how='inner')

breakdown = (alldata[['ProjectCode', 'ProjectBreakdownCode']]
             .groupby('ProjectCode').nunique()
             .drop('ProjectCode', axis=1)
             .reset_index())

def wrangler(X):
## data type conversion  
  X['EstimateOn'] = pd.to_datetime(X['EstimateOn'], format='%d-%b-%y')
  X['StartedOn'] = pd.to_datetime(X['StartedOn'], format='%d-%b-%y')
  X['CompletedOn'] = pd.to_datetime(X['CompletedOn'], format='%d-%b-%y')

## feature engineering
  X['hoursestimatelog'] = np.log1p(X['HoursEstimate'])
  X['daysactual'] = (X['CompletedOn']-X['StartedOn']).dt.days.abs()
  X['estimateonsameday'] = X['EstimateOn']==X['StartedOn']
  ## pandas.DataFrame.merge() behaves awkwardly in a function
  ## so I have to do it in this way
  tmp = (X[['ProjectCode']]
         .merge(right=breakdown, on='ProjectCode', how='left'))
  X['breakdown'] = tmp['ProjectBreakdownCode']
#   tmp = (X[['DeveloperID']].merge(right=performancelevel,
#                                   on='DeveloperID', how='left'))
#   X['performancelevel'] = tmp['performancelevel'].astype(int)

## missing values
  X.replace([np.inf, -np.inf, pd.NaT], np.nan, inplace=True)
wrangler(alldata)
# alldata.to_csv("alldata.csv")

hoursactual_log = np.log1p(alldata['HoursActual'])
hoursestimate_log = np.log1p(alldata['HoursEstimate'])

In [5]:
print(alldata.columns.to_list())

['TaskNumber', 'Summary', 'Priority', 'RaisedByID', 'AssignedToID', 'AuthorisedByID', 'StatusCode', 'ProjectCode', 'ProjectBreakdownCode', 'Category', 'SubCategory', 'HoursEstimate', 'HoursActual', 'DeveloperID', 'DeveloperHoursActual', 'TaskPerformance', 'DeveloperPerformance', 'EstimateOn', 'StartedOn', 'CompletedOn', 'hoursestimatelog', 'daysactual', 'estimateonsameday', 'breakdown']


# Plotly

https://community.plot.ly/t/introducing-plotly-py-4-0-0/25639   
https://plot.ly/python/figure-factory-subplots/  
https://plot.ly/python/distplot/  

Plotly distplot subplot example  
https://nbviewer.jupyter.org/gist/empet/7eaf06b3cb5e488b129bb8df8fdb9b4b   
Formatting ticks  
https://plot.ly/python/v3/tick-formatting/  

In [0]:
# plotly distplot
hist_data = pd.DataFrame([alldata['HoursActual'], alldata['HoursEstimate']])
group_labels = ['Hours Actual', 'Hours Estimated']
fig = ff.create_distplot(hist_data, group_labels, 
                         bin_size=.2, show_rug=False)
  
# Caution: plotly 3.6.1: AttributeError: 'Figure' object has no attribute 'show'
#          fig.show() works for plotly 4.0.0
fig.show()

In [0]:
# plotly distplot
hist_data = pd.DataFrame([hoursactual_log, hoursestimate_log])
path = 'hist_data.csv'
hist_data.to_csv(path)

hist_data = pd.read_csv(path, index_col=0).values
group_labels = ['Hours Actual (log)', 'Hours Estimated (log)']
fig = ff.create_distplot(hist_data, group_labels, 
                         bin_size=0.1, show_rug=False)

fig.update_layout(
    title=go.layout.Title(
        text="Software Development Effort Distribution",
        xref="paper", x=0),
    margin=dict(l=1,r=1,t=30,b=1),  
    legend=dict(x=.85, y=.95),
    xaxis=go.layout.XAxis(
        tickmode = 'array',
        tickvals = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
        ticktext = ['Hours', 'e<br>(2.72 hours)', 'e^2<br>(7.39 hours)', 
                    'e^3<br>(20.09 hours)', 'e^4<br>(54.60 hours)', 'e^5<br>(148.41 hours)', 
                    'e^6<br>(403.43 hours)', 'e^7<br>(1096.63 hours)', 'e^8<br>(2980.96 hours)', 
                    'e^9<br>(8103.08 hours)']),
)

# plotly table

https://plot.ly/python/table/  

In [0]:
url = "https://raw.githubusercontent.com/Nov05/DS-Unit-2-Sprint-4-Project/master/assets/hours_category2.csv"
df = pd.read_csv(url)
df.columns.to_list()

['Unnamed: 0',
 'Category',
 'SubCategory',
 'TaskNumber',
 'HoursEstimateSum',
 'HoursActualSum',
 'HoursEstimateMax',
 'HoursActualMax',
 'HoursEstimateMin',
 'HoursActualMin']

In [0]:
url = "https://raw.githubusercontent.com/Nov05/DS-Unit-2-Sprint-4-Project/master/assets/hours_category2.csv"
path = "assets/hours_category2.csv"
df = pd.read_csv(url)
df = df.round(2)

fig = go.Figure(data=[go.Table(
    header=dict(values=df.columns.to_list()[1:],
                align='left'),
    cells=dict(values=[df['Category'], 
                       df['SubCategory'], 
                       df['TaskNumber'], 
                       df['HoursEstimateSum'],
                       df['HoursActualSum'],
                       df['HoursEstimateMax'],
                       df['HoursActualMax'],
                       df['HoursEstimateMin'],
                       df['HoursActualMin'],
                      ],
               align='left')),
])
fig.update_layout(margin=dict(l=1,r=1,t=30,b=1),
                 )
# fig.show()

# Plotly Scatterplot

https://plot.ly/python/line-and-scatter/#scatter-and-line-plot-with-goscatter  

In [83]:
# read data

fig = go.Figure()
fig.add_trace(go.Scatter(x=hoursactual_log, 
                         y=hoursestimate_log,
                         mode='markers', 
                         hovertemplate="Actual: %{x:.2f}<br>Esitmated: %{y:.2f}",
                         name='effort hours (log)',
                         marker=dict(size=6, opacity=0.9,
                                     color=hoursestimate_log-hoursactual_log,
                                     colorscale='RdBu',
                                     line=dict(width=0.5, color='DarkSlateGrey'),
                                     showscale=True,
                                     reversescale=True,
                                     colorbar=dict(title='Error (log)',
                                                   thickness=5,
                                                   outlinewidth=0,),
                                    ),
                        )
             )
fig.add_trace(go.Scatter(x=[0,8], y=[0,8],
                         mode='lines',
                         line=dict(color='DarkSlateGrey', width=2, dash='dash'),
                         name='perfect estimation'))
fig.update_layout(title='Manuel Estimation',
                  autosize=False, width=550, height=500,
                  margin=dict(l=1,r=1,t=30,b=1),
                  legend=dict(x=.05, y=.95),
                  xaxis=go.layout.XAxis(title=go.layout.xaxis.Title(text="Actual Hours (log)")),
                  yaxis=go.layout.YAxis(title=go.layout.yaxis.Title(text="Estimated Hours (log)")),
                 )